In [1]:
import pandas as pd
import numpy as np
import os
import random

from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [2]:
import os

print(os.getcwd())

/home/wjstjrals417/Dacon/New_drug_development/Seokmin/src/ChemBERTa


In [3]:
from os.path import join
train_path = join('..', '..', '..', 'data', 'total_data', 'train.csv')
test_path = join('..', '..', '..', 'data', 'total_data', 'test.csv')
submission_path = join('..', '..', '..', 'data', 'total_data', 'sample_submission.csv')

## Model Load

In [4]:
model_name = "seyonec/ChemBERTa-zinc-base-v1"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=1)

/home/wjstjrals417/anaconda3/envs/ABSA/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/ChemBERTa-zinc-base-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Load data
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [6]:
# Tokenize SMILES strings with dynamic padding
def tokenize_smiles(smiles):
    return tokenizer(smiles, padding=True, truncation=True, max_length=128, return_tensors='pt')

In [7]:
# Apply tokenization and store as a list
train_tokens = [tokenize_smiles(smiles) for smiles in train_df['Smiles']]
test_tokens = [tokenize_smiles(smiles) for smiles in test_df['Smiles']]

In [8]:
# No scaling applied to IC50 values
y = train_df['IC50_nM'].values

In [9]:
# Split into training and validation sets
train_tokens, val_tokens, y_train, y_val = train_test_split(train_tokens, y, test_size=0.2, random_state=42)

In [10]:
class SMILESDataset(Dataset):
    def __init__(self, tokens, targets=None):
        self.tokens = tokens
        self.targets = targets
    
    def __len__(self):
        return len(self.tokens)
    
    def __getitem__(self, idx):
        item = {key: val.squeeze(0) for key, val in self.tokens[idx].items()}
        if self.targets is not None:
            item['labels'] = torch.tensor(self.targets[idx], dtype=torch.float)
        return item

In [11]:
train_dataset = SMILESDataset(train_tokens, y_train)
val_dataset = SMILESDataset(val_tokens, y_val)
test_dataset = SMILESDataset(test_tokens)

In [12]:
# Use DataCollatorWithPadding to handle dynamic padding
data_collator = DataCollatorWithPadding(tokenizer)

In [13]:
# Prepare training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=500,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)


In [14]:
# Fine-tune the model
trainer.train()

# Predict on the test set
predictions = trainer.predict(test_dataset)

# Save the predictions to a CSV file
output_df = pd.DataFrame({'ID': test_df['ID'], 'IC50_nM': predictions.predictions.squeeze()})
output_df.to_csv('./predictions_chembert_v1.csv', index=False) # 0.6297

/home/wjstjrals417/anaconda3/envs/ABSA/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,4674883.500000
2,No log,4671197.500000
3,No log,4669167.500000
4,No log,4667687.500000
5,No log,4666441.500000
6,8020812.800000,4665314.000000
7,8020812.800000,4664257.000000
8,8020812.800000,4663246.000000
9,8020812.800000,4662261.500000
10,8020812.800000,4661299.000000


/home/wjstjrals417/anaconda3/envs/ABSA/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/wjstjrals417/anaconda3/envs/ABSA/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/wjstjrals417/anaconda3/envs/ABSA/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/wjstjrals417/anaconda3/envs/ABSA/lib/python3.8/site-packages/torch/nn/parallel/_functions.p